In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
import statsmodels.api as sm
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
import plotly.express as px

(CVXPY) Dec 14 03:48:44 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Dec 14 03:48:44 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
industry_returns = pd.read_excel('QPM_Homework_2024_Dataset.xlsx', sheet_name='Industry_Returns')
industry_returns

,Date,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
0,196907,-0.0832,-0.0692,-0.0353,-0.0210,-0.0474,-0.0787,-0.1015,-0.1107,-0.0343,...,-0.0315,-0.1250,-0.1009,-0.0819,-0.0993,-0.0660,-0.1018,-0.1274,-0.0905,-0.1464
1,196908,0.0423,0.0452,0.0544,0.0663,0.0399,0.0642,0.1156,0.0979,0.0477,...,0.0608,0.0559,0.0638,0.0821,0.0898,0.1276,0.0880,0.1506,0.0889,0.0838
2,196909,0.0159,-0.0058,0.0528,-0.0091,0.0412,0.0213,0.0057,-0.0061,-0.0052,...,0.0123,-0.0156,0.0021,-0.0075,-0.0135,-0.0294,0.0708,-0.0379,-0.0254,-0.0667
3,196910,0.1695,0.0797,0.0684,0.1164,0.1242,0.0628,0.1357,0.0759,0.0641,...,0.0350,0.0489,0.1125,0.0625,0.0903,0.1071,0.0571,0.0454,0.1031,0.0370
4,196911,-0.0187,-0.0066,-0.0491,-0.0283,0.0061,-0.0833,-0.0327,-0.0397,-0.0242,...,-0.0198,-0.0791,-0.0683,-0.0408,-0.0190,-0.0899,-0.0490,-0.0623,-0.0591,-0.0225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,202405,0.0427,-0.0144,0.0203,-0.0180,0.0642,0.0061,0.0975,0.1299,0.0020,...,0.0394,0.0036,0.0015,0.0366,-0.0349,0.0295,0.0368,0.0471,0.0472,0.0392
659,202406,-0.0313,-0.0368,-0.0015,-0.0305,0.0103,-0.0365,0.0435,0.0116,-0.0012,...,-0.0836,0.0300,-0.0284,0.0509,0.0060,-0.0062,-0.0024,-0.0087,0.0046,-0.0129
660,202407,0.0511,0.0406,0.0357,0.0304,0.1169,0.0851,-0.0396,0.0315,-0.0107,...,0.1179,0.0020,0.0644,-0.0013,-0.0266,0.0746,0.0844,0.1939,0.0673,0.0590
661,202408,0.0232,0.0509,0.0538,0.0088,0.0796,-0.0192,0.0598,0.0169,0.0630,...,0.0124,0.0245,-0.0121,0.0075,0.0547,0.0220,0.0433,0.0056,0.0098,0.0627


In [3]:
industry_returns['Date'] = pd.to_datetime(industry_returns['Date'], format='%Y%m')
industry_returns['Date'] = industry_returns['Date'].dt.strftime('%Y-%m')
industry_returns.set_index('Date', inplace=True)
industry_returns

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07,-0.0832,-0.0692,-0.0353,-0.0210,-0.0474,-0.0787,-0.1015,-0.1107,-0.0343,-0.1191,...,-0.0315,-0.1250,-0.1009,-0.0819,-0.0993,-0.0660,-0.1018,-0.1274,-0.0905,-0.1464
1969-08,0.0423,0.0452,0.0544,0.0663,0.0399,0.0642,0.1156,0.0979,0.0477,0.0240,...,0.0608,0.0559,0.0638,0.0821,0.0898,0.1276,0.0880,0.1506,0.0889,0.0838
1969-09,0.0159,-0.0058,0.0528,-0.0091,0.0412,0.0213,0.0057,-0.0061,-0.0052,-0.0264,...,0.0123,-0.0156,0.0021,-0.0075,-0.0135,-0.0294,0.0708,-0.0379,-0.0254,-0.0667
1969-10,0.1695,0.0797,0.0684,0.1164,0.1242,0.0628,0.1357,0.0759,0.0641,0.0672,...,0.0350,0.0489,0.1125,0.0625,0.0903,0.1071,0.0571,0.0454,0.1031,0.0370
1969-11,-0.0187,-0.0066,-0.0491,-0.0283,0.0061,-0.0833,-0.0327,-0.0397,-0.0242,-0.0290,...,-0.0198,-0.0791,-0.0683,-0.0408,-0.0190,-0.0899,-0.0490,-0.0623,-0.0591,-0.0225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05,0.0427,-0.0144,0.0203,-0.0180,0.0642,0.0061,0.0975,0.1299,0.0020,0.0491,...,0.0394,0.0036,0.0015,0.0366,-0.0349,0.0295,0.0368,0.0471,0.0472,0.0392
2024-06,-0.0313,-0.0368,-0.0015,-0.0305,0.0103,-0.0365,0.0435,0.0116,-0.0012,-0.1245,...,-0.0836,0.0300,-0.0284,0.0509,0.0060,-0.0062,-0.0024,-0.0087,0.0046,-0.0129
2024-07,0.0511,0.0406,0.0357,0.0304,0.1169,0.0851,-0.0396,0.0315,-0.0107,-0.0236,...,0.1179,0.0020,0.0644,-0.0013,-0.0266,0.0746,0.0844,0.1939,0.0673,0.0590


In [4]:
momentum = industry_returns.rolling(window = 12).sum()
momentum.head(12)

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
momentum_ranks = momentum.rank(axis=1, method="min", ascending=True)
momentum_ranks

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05,14.0,1.0,4.0,2.0,17.0,5.0,32.0,37.0,9.0,7.0,...,33.0,23.0,29.0,38.0,8.0,40.0,24.0,12.0,39.0,34.0
2024-06,12.0,4.0,16.0,3.0,19.0,5.0,37.0,40.0,13.0,2.0,...,25.0,26.0,22.0,44.0,15.0,41.0,31.0,11.0,42.0,36.0
2024-07,17.0,3.0,14.0,2.0,30.0,4.0,26.0,41.0,11.0,1.0,...,28.0,12.0,27.0,43.0,8.0,44.0,35.0,23.0,37.0,39.0


In [6]:
lagged_ranks = momentum_ranks.shift(1)
portfolio_returns = pd.DataFrame(index=industry_returns.index, columns=range(1, 8))

for month in lagged_ranks.index:
    for septile in range(1, 8):
        selected_industries = lagged_ranks.loc[month][(lagged_ranks.loc[month] > (septile - 1) * 7) & 
                                                      (lagged_ranks.loc[month] <= septile * 7)].index
        portfolio_returns.loc[month, septile] = industry_returns.loc[month, selected_industries].mean()

portfolio_returns = portfolio_returns.apply(pd.to_numeric)

In [7]:
portfolio_returns

,1,2,3,4,5,6,7
Date,,,,,,,
1969-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2024-05,0.023771,0.033843,0.030029,0.042057,0.026957,0.044386,0.054729
2024-06,-0.020643,-0.008171,-0.015843,-0.002586,-0.014443,0.011457,-0.043929
2024-07,0.056443,0.052471,0.077671,0.100543,0.060229,0.035086,0.019043


In [8]:
portfolio_returns['Long-Short'] = portfolio_returns[7] - portfolio_returns[1]
portfolio_returns

,1,2,3,4,5,6,7,Long-Short
Date,,,,,,,,
1969-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-05,0.023771,0.033843,0.030029,0.042057,0.026957,0.044386,0.054729,0.030957
2024-06,-0.020643,-0.008171,-0.015843,-0.002586,-0.014443,0.011457,-0.043929,-0.023286
2024-07,0.056443,0.052471,0.077671,0.100543,0.060229,0.035086,0.019043,-0.037400


In [9]:
performance_stats = pd.DataFrame(columns=["Mean", "Std", "Num_Months", "t-statistic", "p-value"])

for column in portfolio_returns.columns:
    returns = portfolio_returns[column].dropna()
    mean_return = returns.mean()
    std_return = returns.std()
    num_months = returns.count()
    
    t_stat, p_value = ttest_1samp(returns, 0)
    
    performance_stats.loc[column] = [mean_return, std_return, num_months, t_stat, p_value]


performance_stats

,Mean,Std,Num_Months,t-statistic,p-value
1,0.006323,0.065557,651.0,2.460897,1.411785e-02
2,0.009110,0.053742,651.0,4.324889,1.765901e-05
3,0.008966,0.050873,651.0,4.496778,8.170344e-06
4,0.011746,0.049501,651.0,6.054173,2.386366e-09
5,0.012103,0.049764,651.0,6.205281,9.723915e-10
6,0.012203,0.049102,651.0,6.341244,4.266927e-10
7,0.014972,0.056621,651.0,6.746514,3.353405e-11
Long-Short,0.008649,0.054859,651.0,4.022486,6.435349e-05


In [10]:
factor_returns = pd.read_excel('QPM_Homework_2024_Dataset.xlsx', sheet_name='Factor_Returns')
factor_returns

,Date,RF,Mkt-RF,SMB,HML,RMW,CMA,Mom
0,196907,0.0053,-0.0700,-0.0341,0.0120,0.0148,0.0191,0.0171
1,196908,0.0050,0.0468,0.0070,-0.0379,0.0113,-0.0402,0.0217
2,196909,0.0062,-0.0298,0.0129,-0.0327,0.0339,-0.0082,0.0252
3,196910,0.0060,0.0506,0.0396,-0.0316,0.0005,-0.0215,-0.0431
4,196911,0.0052,-0.0379,-0.0245,-0.0119,0.0146,0.0030,0.0366
...,...,...,...,...,...,...,...,...
658,202405,0.0044,0.0434,0.0076,-0.0166,0.0298,-0.0307,-0.0002
659,202406,0.0041,0.0277,-0.0437,-0.0331,0.0051,-0.0178,0.0090
660,202407,0.0045,0.0124,0.0828,0.0573,0.0022,0.0043,-0.0242
661,202408,0.0048,0.0161,-0.0365,-0.0113,0.0085,0.0086,0.0479


In [11]:
factor_returns['Date'] = pd.to_datetime(factor_returns['Date'], format='%Y%m')
factor_returns['Date'] = factor_returns['Date'].dt.strftime('%Y-%m')
factor_returns.set_index('Date', inplace=True)
factor_returns

,RF,Mkt-RF,SMB,HML,RMW,CMA,Mom
Date,,,,,,,
1969-07,0.0053,-0.0700,-0.0341,0.0120,0.0148,0.0191,0.0171
1969-08,0.0050,0.0468,0.0070,-0.0379,0.0113,-0.0402,0.0217
1969-09,0.0062,-0.0298,0.0129,-0.0327,0.0339,-0.0082,0.0252
1969-10,0.0060,0.0506,0.0396,-0.0316,0.0005,-0.0215,-0.0431
1969-11,0.0052,-0.0379,-0.0245,-0.0119,0.0146,0.0030,0.0366
...,...,...,...,...,...,...,...
2024-05,0.0044,0.0434,0.0076,-0.0166,0.0298,-0.0307,-0.0002
2024-06,0.0041,0.0277,-0.0437,-0.0331,0.0051,-0.0178,0.0090
2024-07,0.0045,0.0124,0.0828,0.0573,0.0022,0.0043,-0.0242


In [12]:
factor_returns['Long-Short'] = portfolio_returns['Long-Short'].shift(-1)
factor_returns

,RF,Mkt-RF,SMB,HML,RMW,CMA,Mom,Long-Short
Date,,,,,,,,
1969-07,0.0053,-0.0700,-0.0341,0.0120,0.0148,0.0191,0.0171,NaN
1969-08,0.0050,0.0468,0.0070,-0.0379,0.0113,-0.0402,0.0217,NaN
1969-09,0.0062,-0.0298,0.0129,-0.0327,0.0339,-0.0082,0.0252,NaN
1969-10,0.0060,0.0506,0.0396,-0.0316,0.0005,-0.0215,-0.0431,NaN
1969-11,0.0052,-0.0379,-0.0245,-0.0119,0.0146,0.0030,0.0366,NaN
...,...,...,...,...,...,...,...,...
2024-05,0.0044,0.0434,0.0076,-0.0166,0.0298,-0.0307,-0.0002,-0.023286
2024-06,0.0041,0.0277,-0.0437,-0.0331,0.0051,-0.0178,0.0090,-0.037400
2024-07,0.0045,0.0124,0.0828,0.0573,0.0022,0.0043,-0.0242,0.014300


In [13]:
factor_returns = factor_returns.dropna()

In [14]:
X_CAPM = factor_returns['Mkt-RF']
X_FF3 = factor_returns[['Mkt-RF', 'SMB', 'HML']]
X_FF5 = factor_returns[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
X_FF6 = factor_returns[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom']]
y = factor_returns['Long-Short']

X_CAPM = sm.add_constant(X_CAPM)
X_FF3 = sm.add_constant(X_FF3)
X_FF5 = sm.add_constant(X_FF5)
X_FF6 = sm.add_constant(X_FF6)

#CAPM model results
model_CAPM = sm.OLS(y, X_CAPM)
res_CAPM = model_CAPM.fit()
print(res_CAPM.summary())

                            OLS Regression Results                            
Dep. Variable:             Long-Short   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     3.192
Date:                Sat, 14 Dec 2024   Prob (F-statistic):             0.0744
Time:                        15:48:51   Log-Likelihood:                 968.22
No. Observations:                 651   AIC:                            -1932.
Df Residuals:                     649   BIC:                            -1923.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0092      0.002      4.239      0.0

In [15]:
#Fama-French 3 factor model
model_FF3 = sm.OLS(y, X_FF3)
res_FF3 = model_FF3.fit()
print(res_FF3.summary())

                            OLS Regression Results                            
Dep. Variable:             Long-Short   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.717
Date:                Sat, 14 Dec 2024   Prob (F-statistic):              0.162
Time:                        15:48:51   Log-Likelihood:                 969.20
No. Observations:                 651   AIC:                            -1930.
Df Residuals:                     647   BIC:                            -1912.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0093      0.002      4.279      0.0

In [16]:
#Fama-French 5 Factor Model
model_FF5 = sm.OLS(y, X_FF5)
res_FF5 = model_FF5.fit()
print(res_FF5.summary())

                            OLS Regression Results                            
Dep. Variable:             Long-Short   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.544
Date:                Sat, 14 Dec 2024   Prob (F-statistic):              0.174
Time:                        15:48:51   Log-Likelihood:                 970.49
No. Observations:                 651   AIC:                            -1929.
Df Residuals:                     645   BIC:                            -1902.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0091      0.002      4.033      0.0

In [17]:
#Fama-French 6 factor model
model_FF6 = sm.OLS(y, X_FF6)
res_FF6 = model_FF6.fit()
print(res_FF6.summary())

                            OLS Regression Results                            
Dep. Variable:             Long-Short   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.340
Date:                Sat, 14 Dec 2024   Prob (F-statistic):              0.237
Time:                        15:48:51   Log-Likelihood:                 970.66
No. Observations:                 651   AIC:                            -1927.
Df Residuals:                     644   BIC:                            -1896.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0089      0.002      3.893      0.0

In [18]:
portfolio_returns.drop(columns=['Long-Short'], inplace=True)
portfolio_returns

,1,2,3,4,5,6,7
Date,,,,,,,
1969-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2024-05,0.023771,0.033843,0.030029,0.042057,0.026957,0.044386,0.054729
2024-06,-0.020643,-0.008171,-0.015843,-0.002586,-0.014443,0.011457,-0.043929
2024-07,0.056443,0.052471,0.077671,0.100543,0.060229,0.035086,0.019043


In [19]:
portfolio_returns_to_2013 = portfolio_returns.query('Date <= "2013-12"').dropna()
portfolio_returns_to_2013

,1,2,3,4,5,6,7
Date,,,,,,,
1970-07,0.026986,0.078729,0.091457,0.062900,0.080757,0.045871,0.047486
1970-08,0.069000,0.079514,0.053214,0.063029,0.039700,0.025700,0.011586
1970-09,0.252600,0.110500,0.101471,0.070029,0.072686,0.038500,0.043957
1970-10,-0.149000,-0.066286,-0.033343,-0.040386,-0.026500,-0.021343,0.006486
1970-11,0.009814,0.026786,0.065100,0.031057,0.046729,0.048700,0.050229
...,...,...,...,...,...,...,...
2013-08,-0.008643,-0.014543,-0.026829,-0.027600,-0.034343,-0.020214,-0.027057
2013-09,0.002786,0.044229,0.047771,0.049971,0.053957,0.038200,0.074286
2013-10,0.040243,0.044100,0.049186,0.056714,0.039086,0.052371,0.029200


In [20]:
mu = portfolio_returns_to_2013.mean()
S = risk_models.sample_cov(prices = portfolio_returns_to_2013, returns_data = True, frequency = 21)
rf = factor_returns['RF'].loc[:'2013-12'].mean()
# Optimize for maximal Sharpe ratio
ef_opt = EfficientFrontier(mu, S).max_sharpe(risk_free_rate=rf)
ef_opt

OrderedDict([(1, -2.8298555e-09),
             (2, 7.170468e-10),
             (3, 6.026162e-09),
             (4, -2.9047443e-09),
             (5, -8.849811e-10),
             (6, 1.4482247e-09),
             (7, 0.9999999984293964)])

In [21]:
ef_minVol = EfficientFrontier(mu, S).min_volatility()
ef_minVol

OrderedDict([(1, 0.0),
             (2, 0.0539606120646739),
             (3, 0.0806556855353007),
             (4, 0.3484402564658871),
             (5, 0.1055445594330741),
             (6, 0.3693377040976208),
             (7, 0.0420611824034434)])

In [22]:
S_EPO = risk_models.CovarianceShrinkage(prices = portfolio_returns_to_2013, returns_data = True, frequency = 21).shrunk_covariance(delta = 0.2)

ef_opt_shrink = EfficientFrontier(mu, S_EPO).max_sharpe(risk_free_rate=rf)
ef_opt_shrink

OrderedDict([(1, 0.0),
             (2, 0.0),
             (3, 0.0),
             (4, 0.029498087713788),
             (5, 0.1299932345478246),
             (6, 0.172056146357621),
             (7, 0.6684525313807664)])

In [23]:
ef_minVol_shrink = EfficientFrontier(mu, S_EPO).min_volatility()
ef_minVol_shrink

OrderedDict([(1, 0.0),
             (2, 0.1356236521925402),
             (3, 0.1447829074153137),
             (4, 0.2084751884913298),
             (5, 0.1621539120942431),
             (6, 0.2208304032763792),
             (7, 0.128133936530194)])

In [24]:
portfolios = pd.DataFrame()
portfolios['200/100'] = (-1 * portfolio_returns[1]) + (2 * portfolio_returns[7])
portfolios['EquallyWeighted'] = np.mean(portfolio_returns, axis = 1)
portfolios['TangencyPortfolio'] = sum(portfolio_returns[portfolio] * ef_opt[portfolio] for portfolio in portfolio_returns.columns)
portfolios['MinVariance'] = sum(portfolio_returns[portfolio] * ef_minVol[portfolio] for portfolio in portfolio_returns.columns)
portfolios['TangencyPortfolio w/ EPO shrinkage'] = sum(portfolio_returns[portfolio] * ef_opt_shrink[portfolio] for portfolio in portfolio_returns.columns)
portfolios['MinVariance w/ EPO shrinkage'] = sum(portfolio_returns[portfolio] * ef_minVol_shrink[portfolio] for portfolio in portfolio_returns.columns)
portfolios_to_2013 = portfolios.loc[:'2013-12']
portfolios_from_2014 = portfolios.loc['2014-01':]

portfolios_to_2013

,200/100,EquallyWeighted,TangencyPortfolio,MinVariance,TangencyPortfolio w/ EPO shrinkage,MinVariance w/ EPO shrinkage
Date,,,,,,
1969-07,NaN,NaN,NaN,NaN,NaN,NaN
1969-08,NaN,NaN,NaN,NaN,NaN,NaN
1969-09,NaN,NaN,NaN,NaN,NaN,NaN
1969-10,NaN,NaN,NaN,NaN,NaN,NaN
1969-11,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2013-08,-0.045471,-0.022747,-0.027057,-0.024794,-0.026843,-0.025110
2013-09,0.145786,0.044457,0.074286,0.046580,0.064717,0.050036
2013-10,0.018157,0.044414,0.029200,0.050805,0.035283,0.046570


In [25]:
portfolios_from_2014

,200/100,EquallyWeighted,TangencyPortfolio,MinVariance,TangencyPortfolio w/ EPO shrinkage,MinVariance w/ EPO shrinkage
Date,,,,,,
2014-01,0.001071,-0.036724,-0.020986,-0.040646,-0.026544,-0.036900
2014-02,0.093300,0.049202,0.069929,0.050115,0.062205,0.049531
2014-03,-0.034700,0.003110,-0.013586,-0.002598,-0.010767,0.001172
2014-04,-0.035843,-0.000706,0.000171,-0.012503,-0.004508,-0.008393
2014-05,0.033771,0.016549,0.023471,0.019927,0.021528,0.017749
...,...,...,...,...,...,...
2024-05,0.085686,0.036539,0.054729,0.040443,0.048965,0.038891
2024-06,-0.067214,-0.013451,-0.043929,-0.001760,-0.029347,-0.009382
2024-07,-0.018357,0.057355,0.019043,0.064245,0.029561,0.059277


In [26]:
port_metrics_to_2013 = pd.DataFrame({
    'Expected monthly return' : portfolios_to_2013.mean(),
    'Expected volatility of monthly portfolio returns': portfolios_to_2013.std(),
    'Expected monthly Sharpe ratio': (portfolios_to_2013.mean() - rf)/portfolios_to_2013.std(),
    'Annualized Sharpe ratio': (portfolios_to_2013.mean() - rf)/portfolios_to_2013.std() * np.sqrt(12)
})

port_metrics_to_2013

,Expected monthly return,Expected volatility of monthly portfolio returns,Expected monthly Sharpe ratio,Annualized Sharpe ratio
200/100,0.025195,0.093309,0.225013,0.779468
EquallyWeighted,0.010986,0.049718,0.136505,0.472868
TangencyPortfolio,0.015690,0.057399,0.200192,0.693484
MinVariance,0.011902,0.048288,0.159512,0.552566
TangencyPortfolio w/ EPO shrinkage,0.014592,0.052758,0.196992,0.682402
MinVariance w/ EPO shrinkage,0.011803,0.048451,0.156937,0.543646


In [27]:
rf_from_2014 = factor_returns['RF'].loc['2014-01':].mean()

port_metrics_from_2014 = pd.DataFrame({
    'Expected monthly return' : portfolios_from_2014.mean(),
    'Expected volatility of monthly portfolio returns': portfolios_from_2014.std(),
    'Expected monthly Sharpe ratio': (portfolios_from_2014.mean() - rf_from_2014)/portfolios_from_2014.std(),
    'Annualized Sharpe ratio': (portfolios_from_2014.mean() - rf_from_2014)/portfolios_from_2014.std() * np.sqrt(12)
})

port_metrics_from_2014

,Expected monthly return,Expected volatility of monthly portfolio returns,Expected monthly Sharpe ratio,Annualized Sharpe ratio
200/100,0.017246,0.076234,0.210611,0.729577
EquallyWeighted,0.009918,0.047822,0.182485,0.632148
TangencyPortfolio,0.012063,0.053475,0.203321,0.704324
MinVariance,0.010991,0.044862,0.218465,0.756785
TangencyPortfolio w/ EPO shrinkage,0.011740,0.048881,0.215824,0.747635
MinVariance w/ EPO shrinkage,0.010565,0.045620,0.205489,0.711835


In [28]:
cumulative_returns = (1 + portfolios_from_2014).cumprod()
cumulative_returns_max = cumulative_returns.cummax()
drawdowns = (cumulative_returns - cumulative_returns_max) / cumulative_returns_max

In [29]:
fig = px.line(cumulative_returns, title='Cumulative returns of portfolios from 2014', labels={'index': 'Date', 'value': 'Cumulative returns'})
fig.show()

In [30]:
fig = px.line(drawdowns, title = 'Drawdowns of portfolios from 2014', labels = {'index': 'Date', 'value': 'Drawdown'})
fig.show()

In [31]:
CAGR = pd.DataFrame(((1 + portfolios_from_2014).prod() ** (12 / len(portfolios_from_2014))) - 1, columns=['CAGR'])
print('CAGR of portfolios from 2014:')
CAGR.sort_values(by = 'CAGR', ascending = False).style.format('{:.3%}')

CAGR of portfolios from 2014:


,CAGR
200/100,18.772%
TangencyPortfolio,13.563%
TangencyPortfolio w/ EPO shrinkage,13.434%
MinVariance,12.666%
MinVariance w/ EPO shrinkage,12.053%
EquallyWeighted,11.056%


In [32]:
MDD = drawdowns.min()
MDD = pd.DataFrame(MDD, columns=['MDD'])
print('Maximum drawdowns of portfolios')
MDD.sort_values(by = 'MDD').style.format("{:.3%}")

Maximum drawdowns of portfolios


,MDD
200/100,-34.652%
EquallyWeighted,-25.931%
MinVariance,-23.594%
MinVariance w/ EPO shrinkage,-23.393%
TangencyPortfolio w/ EPO shrinkage,-22.041%
TangencyPortfolio,-21.920%


In [37]:
metrics = CAGR.join(MDD, how = 'left')
metrics['Drawdown Ratio'] = (-metrics['CAGR']/metrics['MDD'])
metrics.sort_values(by = 'Drawdown Ratio', ascending = False)

,CAGR,MDD,Drawdown Ratio
TangencyPortfolio,0.135626,-0.219201,0.618731
TangencyPortfolio w/ EPO shrinkage,0.134341,-0.220412,0.609499
200/100,0.187716,-0.346516,0.541725
MinVariance,0.126664,-0.235940,0.536847
MinVariance w/ EPO shrinkage,0.120527,-0.233927,0.515235
EquallyWeighted,0.110562,-0.259314,0.426363
